<a href="https://colab.research.google.com/github/icanardahan/GithubProjelerindeS-r-mTahmini/blob/main/OrjVeriLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Release_Notes_v0_cleaned.xlsx to Release_Notes_v0_cleaned.xlsx


In [ ]:
import io
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re


# fix random seed for reproducibility
numpy.random.seed(7)

In [ ]:
data = pd.read_excel('Release_Notes_v0_cleaned.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 789 entries, 0 to 788
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID            789 non-null    int64         
 1   Project Name  789 non-null    object        
 2   Number        789 non-null    object        
 3   Date          789 non-null    datetime64[ns]
 4   Type Common   789 non-null    object        
 5   Type          789 non-null    object        
 6   Note          789 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 43.3+ KB


In [ ]:
data.loc[data['Type']=='Type Common','Type']

Series([], Name: Type, dtype: object)

In [ ]:
data.Note.value_counts()

develop inform                                                                                                                                                                   50
various secur fix                                                                                                                                                                21
tinker intern work polish rough edg app better                                                                                                                                   10
secur fix                                                                                                                                                                        10
due chang mozilla platform profil store window network share address via drive letter address via unc chat twitter work due api chang twitter com                                 9
                                                                                                    

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250

EMBEDDING_DIM = 64


In [ ]:
X = pd.get_dummies(data['Note'].values)
Y = pd.get_dummies(data['Type']).values
print('Shape of data tensor:', X.shape)

Shape of data tensor: (789, 606)


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(710, 606) (710, 11)
(79, 606) (79, 11)


In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(11, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 10
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/10
10/10 [==============================] - 58s 2s/step - loss: 2.3520 - accuracy: 0.2344 - val_loss: 2.0400 - val_accuracy: 0.2958
Epoch 2/10
10/10 [==============================] - 17s 2s/step - loss: 1.8925 - accuracy: 0.3660 - val_loss: 1.5901 - val_accuracy: 0.2958
Epoch 3/10
10/10 [==============================] - 17s 2s/step - loss: 1.6573 - accuracy: 0.3029 - val_loss: 1.5521 - val_accuracy: 0.2958
Epoch 4/10
10/10 [==============================] - 17s 2s/step - loss: 1.5595 - accuracy: 0.4010 - val_loss: 1.4962 - val_accuracy: 0.2958
Epoch 5/10
10/10 [==============================] - 17s 2s/step - loss: 1.6089 - accuracy: 0.3732 - val_loss: 1.5183 - val_accuracy: 0.2958
Epoch 6/10
10/10 [==============================] - 17s 2s/step - loss: 1.6494 - accuracy: 0.3559 - val_loss: 1.5348 - val_accuracy: 0.2958
Epoch 7/10
10/10 [==============================] - 17s 2s/step - loss: 1.6051 - accuracy: 0.3651 - val_loss: 1.5301 - val_accuracy: 0.2958


In [ ]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))


3/3 [==============================] - 0s 142ms/step - loss: 1.4956 - accuracy: 0.4051
Test set
  Loss: 1.496
  Accuracy: 0.405
